In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import cv2
from skimage.io import imread,imshow
import os
import random
from tqdm import tqdm
from skimage.transform import resize
from sklearn.model_selection import train_test_split

## Performance of vanillla model with image preprocessing

In [38]:
TRAIN_PATH = "dataset/2d_slices/data_for_model/train/pre_processed_images"
TRAIN_MASK_PATH = "dataset/2d_slices/data_for_model/train/masks"

In [4]:
TEST_PATH = "dataset/2d_slices/data_for_model/test/pre_processed_images"
TEST_MASK_PATH = "dataset/2d_slices/data_for_model/test/masks"

In [29]:
original_path = "dataset/2d_slices/original_image_for_model/test/imgs"

In [30]:
train_ids = next(os.walk(TRAIN_PATH))[2]
mask_train_ids = next(os.walk(TRAIN_MASK_PATH))[2]
original_ids = next(os.walk(original_path))[2]
test_ids = next(os.walk(TEST_PATH))[2]
mask_test_ids = next(os.walk(TEST_MASK_PATH))[2]

In [31]:
IMG_WIDTH = 512
IMG_HEIGHT = 512
IMG_CHANNELS = 1

In [39]:
X_train = np.zeros((len(train_ids),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.float16)
Y_train=np.zeros((len(mask_train_ids),IMG_HEIGHT,IMG_WIDTH,1),dtype = np.float16)
W_test = np.zeros((len(original_ids),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.float16)
X_test= np.zeros((len(test_ids),IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS),dtype=np.float16)
Y_test= np.zeros((len(mask_test_ids),IMG_HEIGHT,IMG_WIDTH,1),dtype = np.float16)

In [40]:
for n,id_ in tqdm(enumerate(train_ids),total=len(train_ids)):
    path=TRAIN_PATH+"/"+id_
    img = np.load(path)
    img = resize(img,(IMG_HEIGHT,IMG_WIDTH),mode="constant",preserve_range=True)
    X_train[n]=np.expand_dims(img, axis=2)

100%|██████████████████████████████████████████████████████████████████████████████| 1499/1499 [01:44<00:00, 14.37it/s]


In [43]:
for n,id_ in tqdm(enumerate(mask_train_ids),total=len(mask_train_ids)):
    path=TRAIN_MASK_PATH+"/"+id_
    mask = np.load(path)
    mask = resize(mask,(IMG_HEIGHT,IMG_WIDTH),mode="constant",preserve_range=True)
    Y_train[n]=np.expand_dims(mask, axis=2)

100%|██████████████████████████████████████████████████████████████████████████████| 1499/1499 [01:48<00:00, 13.86it/s]


In [44]:
for n,id_ in tqdm(enumerate(original_ids),total=len(original_ids)):
    path=original_path+"/"+id_
    img = np.load(path)
    img = resize(img,(IMG_HEIGHT,IMG_WIDTH),mode="constant",preserve_range=True)
    W_test[n]=np.expand_dims(img, axis=2)

100%|████████████████████████████████████████████████████████████████████████████████| 177/177 [00:05<00:00, 30.11it/s]


In [45]:
for n,id_ in tqdm(enumerate(test_ids),total=len(test_ids)):
    path=TEST_PATH+"/"+id_
    img = np.load(path)
    img = resize(img,(IMG_HEIGHT,IMG_WIDTH),mode="constant",preserve_range=True)
    X_test[n]=np.expand_dims(img, axis=2)

100%|████████████████████████████████████████████████████████████████████████████████| 177/177 [00:11<00:00, 15.84it/s]


In [46]:
for n,id_ in tqdm(enumerate(mask_test_ids),total=len(mask_test_ids)):
    path=TEST_MASK_PATH+"/"+id_
    mask = np.load(path)
    mask = resize(mask,(IMG_HEIGHT,IMG_WIDTH),mode="constant",preserve_range=True)
    Y_test[n]=np.expand_dims(mask, axis=2)

100%|████████████████████████████████████████████████████████████████████████████████| 177/177 [00:11<00:00, 14.95it/s]


In [64]:
model_with_image_augmentation=tf.keras.models.load_model("lungnodule_augmentation.h5",compile=False)

In [85]:
overall_dice_score = 0
for i in range(0,len(X_test)):  
    c = np.expand_dims(X_test[i], axis=0)
    y_pred=model_with_image_augmentation.predict(c)
    y_pred_thresholded = y_pred > 0.7
    d = np.squeeze(y_pred_thresholded, axis=0)
    dice_score = (2*(d*Y_test[i]).sum()) / ((d+Y_test[i]).sum() + 1e-12)
    overall_dice_score = overall_dice_score + dice_score
#     fig = plt.figure(figsize=(15, 15))
#     fig.add_subplot(1, 4, 1)
#     plt.imshow(W_test[i],cmap='gray')
#     plt.title("2D slice from CT scan")
#     fig.add_subplot(1, 4, 2)
#     plt.imshow(X_test[i],cmap='gray')
#     plt.title("2D slice after image_preprocssing")
#     fig.add_subplot(1, 4, 3)
#     plt.imshow(Y_test[i],cmap='gray')
#     plt.title("Ground truth mask of nodule")
#     fig.add_subplot(1, 4, 4)
#     plt.imshow(d,cmap='gray')
#     plt.title("Predicted mask of nodule")
    #plt.savefig("Evaluated_images/with_augmentation/sample"+ str(i)+" "+'dice_score'+" "+str(int(np.ceil(dice_score*100))), bbox_inches='tight')
print("Dice score on test dataset is: ", overall_dice_score)
#plt.savefig('Evaluated_images/vanilla_with_prepreprocess/sample'+ str(i), bbox_inches='tight')   

1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 35ms/step
Dice score on test dataset is:  71.92690177712345
